# Scraping Ado

This script shows how to scrape data from [LombardiaCanestro](https://lombardia.italiacanestro.it/). In this case, scrape the **Promozione - Girone E League**. Here the list of all Teams:

* [Aurora Trezzo](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=482).
* [Posal Sesto San Giovanni](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=483).
* [Ado San Benedetto Milano](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=484).
* [CGB Brugherio](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=485).
* [Azzurri Niguardese](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=486).
* [Pallacanestro Carugate](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=487).
* [CBBA Olimpia Cologno](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=488).
* [Cesano Seveso](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=489).
* [Inzago Basket](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=490).
* [OSAL Novate](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=491).
* [Basket Ajaccio 1988](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=492).
* [Social OSA](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=493).
* [Basket San Rocco 2013 Seregno](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=494).
* [Ciesse Freebasket Milano](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=495).
* [ACLI Trecella](https://lombardia.italiacanestro.it/Maschile/Squadra?id=42&tid=496).

## Python Script

The following cells shows the scrape script from the LombardiaCanestro Site. Scrape data:

* *All games*, with this link: `https://lombardia.italiacanestro.it/Maschile/Partita?id=`***`<id_game>`***
* *Standings*, with this link: `https://lombardia.italiacanestro.it/Maschile/Calendario?id=`***`42`***
* *Rosters*, with this link: `https://lombardia.italiacanestro.it/Maschile/Roster?id=`***`42`***

### Libraries

Used libraries:

* `pandas`.
* `pycelonis`.

In [1]:
import pandas as pd
import numpy as np
from   urllib.request import Request, urlopen
from   tqdm import tqdm

#-- Celonis
from pycelonis import get_celonis
celonis = get_celonis(
    base_url = "alberto-filosa-protiviti-it.training.celonis.cloud",
    api_token = "NzQ4Mzg3YjctNzkzNy00ZTFhLWE5ZTUtN2Y5NDk0MGVhYWJiOnlHK2xYb3NKRHpwTitGU053NUxOT2ZDZFZOUllKaXNsNWlUeGFwVnJ0UTc3",
    key_type = 'USER_KEY'
)

[2023-06-04 09:04:32,953] WARNING: Your PyCelonis Version 2.0.0 is outdated (Newest Version: 2.2.0). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis --upgrade


[2023-06-04 09:04:33,159] INFO: Initial connect successful! PyCelonis Version: 2.0.0
[2023-06-04 09:04:33,356] INFO: `package-manager` permissions: ['EDIT_ALL_SPACES', 'MANAGE_PERMISSIONS', 'CREATE_SPACE', 'DELETE_ALL_SPACES']
[2023-06-04 09:04:33,357] INFO: `workflows` permissions: ['EDIT_AGENTS', 'VIEW_AGENTS', 'REGISTER_AGENTS', 'MANAGE_PERMISSIONS']
[2023-06-04 09:04:33,357] INFO: `task-mining` permissions: ['EDIT_CLIENT_SETTINGS', 'EDIT_USERS']
[2023-06-04 09:04:33,358] INFO: `action-engine` permissions: ['CREATE_PROJECTS', 'MANAGE_SKILLS', 'ACCESS_ALL_PROJECTS', 'MY_INBOX']
[2023-06-04 09:04:33,359] INFO: `team` permissions: ['MANAGE_AUDIT_LOGS', 'MANAGE_SSO_SETTINGS', 'USE_AUDIT_LOGS_API', 'MANAGE_ADOPTION_VIEWS', 'MANAGE_GENERAL_SETTINGS', 'MANAGE_GROUPS', 'MANAGE_APPLICATIONS', 'USE_STUDIO_ADOPTION_API', 'MANAGE_LOGIN_HISTORY', 'MANAGE_LICENSE_SETTINGS', 'USE_LOGIN_HISTORY_API', 'MANAGE_MEMBERS', 'MANAGE_UPLINK_INTEGRATIONS', 'MANAGE_PERMISSIONS', 'MANAGE_ADMIN_NOTIFICATIONS',

### Scraping Webpage

The `scraping_webpage` function takes in input the URL of the page to scrape the data (treated as *string*). It returns a DataFrame of the URL table.

This function is nested in the following tables:

* `scraping_table`.
* `scraping_players`.
* `scraping_standings`.

In [2]:
def scraping_webpage(lv_url, link = None):
        
    #-- Get URL
    req = Request(lv_url, headers = {'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    #-- Get Table in HTML
    df_scraped = pd.read_html(webpage, extract_links = link)
    
    return df_scraped

### Games Scraping

<!-- Inserire cosa fa la funzione -->

In [3]:
def scraping_table(ls_id_game):
    #-- Disable chained assignments
    pd.options.mode.chained_assignment = None 

    #-- Read the existing scraped data from the CSV file (if it exists)
    try:
        existing_data = pd.read_csv('scraped_data.csv')
    except FileNotFoundError:
        existing_data = pd.DataFrame()

    #-- Initialize an empty DataFrame to store the scraped data
    df_all_games = pd.DataFrame()

    for game in tqdm(ls_id_game):
        
        #-- Check if any data already exists
        if len(existing_data) > 0:
            #-- Check if the ID has already been scraped
            if len(existing_data[existing_data['id_gara'] == game]) > 0:
                continue

        lv_url = f"https://lombardia.italiacanestro.it/Maschile/Partita?id={game}"
        
        #-- Call the scraping_webpage function to scrape the Team Table
        df_single_game = scraping_webpage(lv_url)[0]
        
        #-- If not played yet, move to the next game
        if len(df_single_game.index) < 5:
            continue
        
        #-- List of Teams
        ls_teams = df_single_game.loc[df_single_game[1] == "PTS"][0]
        
        #-----------------------
        #-- Data Manipulation --
        #-----------------------
        
        #-- Drop NAs (in a single URL there is one single Table to identify the Teams)
        df_single_game_nona = df_single_game.dropna(how = "all")
        
        #-- Add Columns in the DataFrame
        df_single_game_nona["Squadra"]    = [ls_teams[0] if ls_teams.index[1] > row else ls_teams[ls_teams.index[1]] for row in range(0, df_single_game_nona.shape[0])]
        df_single_game_nona["Squadra"]    = df_single_game_nona["Squadra"].str.title()
        df_single_game_nona["Avversario"] = [ls_teams[ls_teams.index[1]] if ls_teams.index[1] > row else ls_teams[0] for row in range(0, df_single_game_nona.shape[0])]
        df_single_game_nona["Avversario"] = df_single_game_nona["Avversario"].str.title()
        df_single_game_nona["Partita"]    = ["C" if ls_teams.index[1] > row else "T" for row in range(0, df_single_game_nona.shape[0])]
        df_single_game_nona["id_gara"]    = game
        
        #-- Remove Header Rows (if they have PTS in the first column)
        df_single_game_end = df_single_game_nona[df_single_game_nona[1] != 'PTS']
        df_single_game_end.columns = ["giocatore", "punti_totali", "tiri_liberi",
                                      "due_punti", "tre_punti", "squadra",
                                      "avversario", "partita","id_gara"]
        
        df_single_game_end["giocatore"] = df_single_game_end["giocatore"].str.title()
        
        #-- Concatenate games
        df_all_games = pd.concat([df_all_games, df_single_game_end])
        
        #-- Add the scraped data to the existing data DataFrame
        existing_data = pd.concat([existing_data, df_single_game_end])
    
    #-- Write the updated data to the CSV file
    existing_data.to_csv('scraped_data.csv', index = False)
    
    return df_all_games

### Rosters

In [4]:
def scraping_players(id_roster):
    
    #-- Iniziate Empty DataFrame. Then, will be inserted for each ig_game the Game Result
    df_all_players = pd.DataFrame()
    
    lv_url = f"https://lombardia.italiacanestro.it/Maschile/Roster?id={id_roster}"
    
    #-- Call the scraping_webpage function to scrape the Team Table
    df_single_team = scraping_webpage(lv_url)
    
    for team in tqdm(df_single_team):
    
        team["Squadra"] = team.columns[1].title()
        team.columns = ["Numero", "Giocatore", "Squadra"]
        df_all_players = pd.concat([df_all_players, team], axis = 0, ignore_index = True)
    
    df_all_players["Giocatore"] = df_all_players["Giocatore"].str.title()
    
    return df_all_players

### Schedule Scraping

The `scraping_schedule` takes in input the following variables:

* `id_championship`: the identifier of the Championship.
* `ls_turn`: the identifier of the turns (Andata - 2 and Ritorno - 3 in the URL)
* `ls_round`: the number of the single round (from 1 to 15).

It returns the Dataframe containing all Schedule of the Championship.

In [5]:
def scraping_schedule(id_championship, ls_turn, ls_round):
    
    #-- Disable chained assignments
    pd.options.mode.chained_assignment = None 
    
    #-- Iniziate Empty DataFrame. Then, will be inserted for each ig_game the Game Result
    df_all_schedules  = pd.DataFrame()
    
    for turns in ls_turn:
        print(f"Scraping Round {turns - 1} ...")
        
        for rounds in tqdm(ls_round):
            
            lv_url = f"https://lombardia.italiacanestro.it/Maschile/Calendario?handler=Change&ChampionshipId={id_championship}&TurnId={turns}&TurnNm={rounds}"
            
            df_single_round = scraping_webpage(lv_url, link = "body")[0]
            df_single_round_tidy = pd.DataFrame()
            df_single_round_tidy["Data"] = df_single_round[0][df_single_round.index % 4 == 0].reset_index(drop = True).str.get(0).str.title()
            df_single_round_tidy["Squadra"] = df_single_round[0][(df_single_round.index - 1) % 4 == 0].reset_index(drop = True).str.get(0).str.replace(r'\([^)]*\)', '', regex = True).str.title()
            df_single_round_tidy["Id_Gara"] = df_single_round[0][(df_single_round.index + 2) % 4 == 0].reset_index(drop = True).str.get(1).str.extract(r'(\d+)').astype(int)
            df_single_round_tidy["Risultato"] = df_single_round[1][(df_single_round.index - 1) % 4 == 0].reset_index(drop = True).str.get(0)
            df_single_round_tidy["Avversario"] = df_single_round[2][(df_single_round.index - 1) % 4 == 0].reset_index(drop = True).str.get(0).str.replace(r'\([^)]*\)', '', regex = True).str.title()
            df_single_round_tidy["Girone"] = turns - 1
            df_single_round_tidy["Turno"] = rounds
            df_single_round_tidy.head()
            
            #-- Concat Games
            df_all_schedules = pd.concat([df_all_schedules, df_single_round_tidy])
            
    return df_all_schedules

### Standings

In [6]:
def scraping_standings(id_calendar):
    
    lv_url = f"https://lombardia.italiacanestro.it/Maschile/Calendario?id={id_calendar}"

    #-- Call the scraping_webpage function to scrape the Standing Table
    df_standings = scraping_webpage(lv_url)[1]

    #-- Data String Manipulation
    df_standings["CLASSIFICA"] = df_standings["CLASSIFICA"].str.title()

    df_standings.columns = ["posizione", "squadra", "punti", "partite_giocate",
                            "vittorie",  "sconfitte", "punti_fatti", "punti_subiti"]

    return df_standings

## Upload to Celonis

| DataFrame Name     | SQL Table Name      |
|--------------------|---------------------|
| `df_all_games`     | `PR_DATA_GAMES`     |
| `df_standing`      | `PR_DATA_STANDINGS` |
| `df_all_players`   | `PR_PLAYERS_NAME`   |
| `df_all_schedules` | `PR_SCHEDULES`      |


In [7]:
%%time

#-----------------------#
#-- Scraping Girone E --#
#-----------------------#

print("Downloading All Games ...")
df_all_games   = scraping_table(np.arange(5508, 5714))

print("Downloading All Players ...")
df_all_players = scraping_players(42)

print("Downloading Standins ...")
df_standing    = scraping_standings(42)

print("Downloading All Schedule ...")
df_all_schedules = scraping_schedule(42, np.arange(2, 4), np.arange(1, 16))

100%|██████████| 206/206 [00:45<00:00,  4.56it/s]


100%|██████████| 15/15 [00:00<00:00, 1192.04it/s]


Scraping Round 1 ...


100%|██████████| 15/15 [00:05<00:00,  2.97it/s]


Scraping Round 2 ...


100%|██████████| 15/15 [00:05<00:00,  2.53it/s]

CPU times: user 6.97 s, sys: 395 ms, total: 7.36 s
Wall time: 56.7 s


%%time

```py
#-----------------------#
#-- Scraping Girone G --#
#-----------------------#

print("Downloading All Games ...")
df_all_games   = scraping_table(np.arange(5958, 6196))

print("Downloading All Players ...")
df_all_players = scraping_players(44)

print("Downloading Standins ...")
df_standing    = scraping_standings(44)

print("Downloading All Schedule ...")
df_all_schedules = scraping_schedule(44, np.arange(2, 4), np.arange(1, 16))
```

In [8]:
#-- Selecting Data Pool, Data Model and Data Job
data_pool = celonis.data_integration.get_data_pool("26a8fa87-21b1-4850-9447-48c2e6a171fc")
data_model = data_pool.get_data_model("9fb8576b-a8f6-4f71-9cb9-4722bafa7a92")
print(f"Selected the '{data_pool.name}' Data Pool and the '{data_model.name}' Data Model")

data_job = data_pool.get_job("f9300adf-cde5-43d8-bc44-eca7b355fda1")

Selected the 'Basket - Scraping Data' Data Pool and the 'Data Model - Promozione - Girone E' Data Model


In [9]:
dict_df_games = {
    "PR_DATA_GAMES":     df_all_games,
    "PR_DATA_STANDINGS": df_standing,
    "PR_PLAYERS_NAME":   df_all_players,
    "PR_SCHEDULES":      df_all_schedules
}

for lv_sql_table, lv_data_frame in dict_df_games.items():
    
    print(f"Uploading of the '{lv_sql_table}' Table from Python to Celonis: \n")
    
    data_pool.create_table(table_name     = lv_sql_table,
                           df             = lv_data_frame,
                           drop_if_exists = True,
                           force          = True)
    
    print("Upload of the Table Completed!")
    print("_" * 45, "\n \n")

Uploading of the 'PR_DATA_GAMES' Table from Python to Celonis: 

[2023-06-04 09:05:43,596] INFO: Successfully created data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'
[2023-06-04 09:05:43,597] INFO: Add data frame as file chunks to data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'


  0%|          | 0/1 [00:00<?, ?it/s]

[2023-06-04 09:05:44,233] INFO: Successfully upserted file chunk to data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'
[2023-06-04 09:05:44,406] INFO: Successfully triggered execution for data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'
[2023-06-04 09:05:44,407] INFO: Wait for execution of data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'


0it [00:00, ?it/s]

[2023-06-04 09:05:58,784] INFO: Successfully created table 'PR_DATA_GAMES' in data pool
[2023-06-04 09:05:58,941] INFO: Successfully deleted data push job with id '0101c0c5-2286-4c92-98ef-0d7f15410f4a'
Upload of the Table Completed!
_____________________________________________ 
 

Uploading of the 'PR_DATA_STANDINGS' Table from Python to Celonis: 

[2023-06-04 09:06:01,797] INFO: Successfully created data push job with id '2510ab1e-c3d3-42da-83ad-444942244acc'
[2023-06-04 09:06:01,798] INFO: Add data frame as file chunks to data push job with id '2510ab1e-c3d3-42da-83ad-444942244acc'


...

[2023-06-04 09:06:38,578] INFO: Successfully created table 'PR_PLAYERS_NAME' in data pool
[2023-06-04 09:06:39,033] INFO: Successfully deleted data push job with id '7cc773ce-8e00-4169-96b4-cd054caf451b'
Upload of the Table Completed!
_____________________________________________ 
 

Uploading of the 'PR_SCHEDULES' Table from Python to Celonis: 

[2023-06-04 09:06:43,489] INFO: Successfully created data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'
[2023-06-04 09:06:43,490] INFO: Add data frame as file chunks to data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'


  0%|          | 0/1 [00:00<?, ?it/s]

[2023-06-04 09:06:43,763] INFO: Successfully upserted file chunk to data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'
[2023-06-04 09:06:43,925] INFO: Successfully triggered execution for data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'
[2023-06-04 09:06:43,926] INFO: Wait for execution of data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'


0it [00:00, ?it/s]

[2023-06-04 09:06:58,401] INFO: Successfully created table 'PR_SCHEDULES' in data pool
[2023-06-04 09:06:58,561] INFO: Successfully deleted data push job with id 'eb8f8662-73a9-468d-a719-87bef133f8b6'
Upload of the Table Completed!
_____________________________________________ 
 



In [10]:
data_job.name
data_job.execute()

[2023-06-04 09:07:02,102] INFO: Successfully started execution for job with id 'f9300adf-cde5-43d8-bc44-eca7b355fda1'
[2023-06-04 09:07:02,103] INFO: Wait for execution of job with id 'f9300adf-cde5-43d8-bc44-eca7b355fda1'


0it [00:00, ?it/s]

In [ ]:
data_model.reload()